# Bolt Tracking

The goal of the project is to detect, track and count the number of bolts on a moving conveyor belt.
This project uses YOLOv8.

# Training

In [ ]:
# Access google drive and change directory to project folder
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/DadepardazAfagh/Bolts

!git clone https://github.com/ultralytics/ultralytics.git

%cd ultralytics/

!pip install -r requirements.txt
!pip install ultralytics
!pip install roboflow

In [ ]:
# DATASET: https://universe.roboflow.com/ml-datasets/bolts-on-conveyor-belt

from roboflow import Roboflow
rf = Roboflow(api_key="<YOUR KEY>")
project = rf.workspace("ml-datasets").project("bolts-on-conveyor-belt")
dataset = project.version(1).download("yolov8")

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8m.pt")  # load a pretrained model (recommended for training)

# Train the model
model.train(data="/content/gdrive/MyDrive/DadepardazAfagh/Bolts/ultralytics/Bolts-on-Conveyor-Belt-1/data.yaml", device='0', batch=16, epochs=30, imgsz=640)

# Implementation

In [2]:
import os
import cv2
import imageio
from ultralytics import YOLO

# A function to show a progress bar
from IPython.display import HTML, display
def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 50%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

In [ ]:
model = YOLO('bolts.pt')
tracker = 'botsort.yaml'
read_dir = 'final_test_bolts'
save_gif = 'bolts.gif'

count = set()
frames = list()

results = model.track(read_dir, device='0', tracker=tracker)

out = display(progress(0, len(results) - 1), display_id=True)

for result in results:
  out.update(progress(results.index(result), len(results) - 1))

  image_path = result.path
  image = cv2.imread(image_path)

  if image is None:
    continue

  scale_percent = 20 # percent of original size
  width = int(image.shape[1] * scale_percent / 100)
  height = int(image.shape[0] * scale_percent / 100)
  dim = (width, height)
  # resize image
  image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)

  width = int(image.shape[1])
  height = int(image.shape[0])

  half = height // 2

  xywhs = result.boxes.xywhn
  ids = result.boxes.id

  image = cv2.line(image, (0, half), (width, half), (0, 255, 255), 2) 

  for (x, y, w, h), id in zip(xywhs, ids):
    x, y, id = int(x * width), int(y * height), int(id)
    if y >= half:
      color = (0, 0, 255)
    else:
      color = (0, 255, 0)
      count.add(id)
    image = cv2.circle(image, (x, y), 3, color, -1)
  
  image = cv2.putText(image, f'Count: {len(count)}', (0, 18), fontFace=2, 
                   fontScale=0.5, color=(0, 255, 0), thickness=1)
  
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  frames.append(image.copy())


imageio.mimsave(save_gif, frames, fps=15)

In [71]:
# Check resuts
from IPython.display import Image
Image(save_gif)